In [1]:
from rocketcea.cea_obj_w_units import CEA_Obj
from rocketcea.cea_obj import add_new_fuel
from scipy.optimize import minimize
import numpy as np

In [2]:
allvectors = []               # array for all design vecs, global variable
allobjfun = []                # array for tracking objective function evaluations

In [3]:
def objective(var, cons):
    return (var/cons)**2 / 2
def exterior(var, cons, good_if_less_than=False):
    if good_if_less_than:
        return np.max([0, var/cons - 1])**2 / 2
    else:
        return np.max([0, -(var/cons - 1)])**2 / 2

In [18]:
def get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio):
    ipa_wt = min(ipa_wt, 100)
    card_str = '''
    fuel C3H8O-2propanol C 3 H 8 O 1    wt%=''' + str(ipa_wt) + '''
    h,cal=-65133.     t(k)=298.15   rho,g/cc=0.786
    fuel water H 2 O 1  wt%=''' + str(100 - ipa_wt) + '''
    h,cal=-68308.  t(k)=298.15 rho,g/cc=0.9998
    '''
    add_new_fuel('LV4_Fuel', card_str)
    PROPELLANT = CEA_Obj(oxName='LOX', fuelName='LV4_Fuel',
                        pressure_units='Pa', temperature_units='K', cstar_units='m/s',
                         density_units='kg/m^3', isp_units='sec', specific_heat_units='J/kg-K')
    isp = PROPELLANT.get_Isp(Pc=p_ch, MR=of_ratio, eps=exp_ratio)
    IspVac, Cstar, Tc, MW, gamma = PROPELLANT.get_IvacCstrTc_ChmMwGam(Pc=p_ch, MR=of_ratio, eps=exp_ratio)
    return [isp, IspVac, Cstar, Tc, MW, gamma]

def cost(x):
    ipa_wt, of_ratio, p_ch, exp_ratio = x
    isp, IspVac, Cstar, Tc, MW, gamma = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio)
    
    merit = -objective(IspVac, 230) + exterior(p_ch, 3.447e6, True) + exterior(exp_ratio, 10, True)
    return merit

def opt(x0):
    res = minimize(cost, x0, method='nelder-mead', options={'disp': True, 'adaptive':True, 'xatol': 0.01, 'fatol': 0.01})
    x = res.x # feed optimal design vec into next iteration
    return x

def run():
    x = opt([64.8, 1.3, 2413166, 5.988])
    #x = [64.8, 1.3, 2413166, 5.988]
    ipa_wt, of_ratio, p_ch, exp_ratio = x
    ipa_wt = min(ipa_wt, 100)
    isp, IspVac, Cstar, Tc, MW, gamma = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio)
    print('IPA Wt %: ', ipa_wt)
    print('OF: ', of_ratio)
    print('P_ch (Pa): ', p_ch)
    print('Expansion ratio: ', exp_ratio)
    print()
    print('Vacuum ISP (s): ', IspVac)
    print('Chamber Temp (K): ', Tc)
    print('Molar Wt (1/n): ', MW)
    print('Spec Heat: ', gamma)
    return x
    

In [19]:
run()

reading cea isp data files for LOX / LV4_Fuel 100 times
reading cea isp data files for LOX / LV4_Fuel 200 times
reading cea isp data files for LOX / LV4_Fuel 300 times
reading cea isp data files for LOX / LV4_Fuel 400 times
reading cea isp data files for LOX / LV4_Fuel 500 times
Optimization terminated successfully.
         Current function value: -0.976021
         Iterations: 231
         Function evaluations: 502
IPA Wt %:  100
OF:  1.8999541338024426
P_ch (Pa):  3492585.6485239915
Expansion ratio:  11.373070500199981

Vacuum ISP (s):  322.9080380872478
Chamber Temp (K):  3427.294809755733
Molar Wt (1/n):  23.10157522662069
Spec Heat:  1.128944232888362


array([1.08775356e+02, 1.89995413e+00, 3.49258565e+06, 1.13730705e+01])